In [ ]:
from pyspark.sql.functions import *

In [ ]:
# File location and type
file_location = "/FileStore/tables/fraudTrain.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
train = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(train)

In [ ]:
df=train.union(test)
df.count()

In [ ]:
#show database
spark.catalog.listDatabases()

In [ ]:
#show database with sql
spark.sql("show databases")

In [ ]:
#spark.catalog.listTables(db_name)
spark.catalog.listTables()

In [ ]:
"""%sql
DROP TABLE fraudTest_csv;
DROP TABLE fraudTrain_csv;"""

In [ ]:
spark.sql('show tables from default').show() #global_temp

In [ ]:
for col in df.dtypes:

    print(col[0]+" , "+col[1])

In [ ]:
for field in df.schema.fields:
    print(field.name +" , "+str(field.dataType))

In [ ]:
train.printSchema()

In [ ]:
from pyspark.sql.types import *
# get string
str_cols = [f.name for f in df.schema.fields if isinstance(f.dataType, StringType)]
# ['colc']

# or double
num_cols = [f.name for f in df.schema.fields if isinstance(f.dataType, (DoubleType,IntegerType))]
# ['colb']

dbl= [f.name for f in df.schema.fields if isinstance(f.dataType, DoubleType)]
# ['colb']

In [ ]:
from pyspark.sql.functions import isnull, when, count, col
nacounts = df.select([count(when(isnull(c), c)).alias(c) for c in df.columns]).toPandas()
nacounts

In [ ]:
# Generic solution 
amount_missing_df = train[num_cols].select([(count(when(isnan(c) | col(c).isNull(), c))/count(lit(1))).alias(c) for c in train[num_cols].columns])
amount_missing_df.show()

In [ ]:
fraud=df.filter(df.is_fraud==1)

In [ ]:
fraud.show(n=1, truncate=False, vertical=True)

In [ ]:
print((df.count(), len(df.columns)))
print((fraud.count(), len(fraud.columns)))

In [ ]:
df.groupBy('category').count().orderBy('count', ascending=False).show()

In [ ]:
cat_fraud=df.groupby("is_fraud")\
   .pivot("category")\
   .count()\
   .display()

In [ ]:
df.groupBy('is_fraud').agg(avg('amt').alias('average amount')).display()

In [ ]:
df.groupBy('is_fraud') .agg(avg("amt").alias("avg"), \
         max("amt").alias("max") \
                              ).show()                       
    

In [ ]:
df.groupby(['is_fraud','state']).count().orderBy(['is_fraud','count'], ascending=[False,False]).show()

In [ ]:
import pyspark.sql.functions as F
df.select(F.countDistinct("_c0")).show()

In [ ]:
from pyspark.sql.window import Window

df.groupBy(['is_fraud','state']).count()\
.withColumn('total', F.sum('count').over(Window.partitionBy()))\
.withColumn('percentage',F.col('count')/F.col('total'))\
.orderBy(['is_fraud','percentage'], ascending=[False,False])\
.show(5)

In [ ]:
df.groupby(['is_fraud','state']).agg(
    (F.count('_c0')).alias('count'),
    (F.count('_c0') / df.count()).alias('percentage')
).orderBy(['is_fraud','percentage'], ascending=[False,False]).show()

In [ ]:
ny=df.filter(df.state=='NY')
ny.summary().display()

In [ ]:
names = df[['first','last']].distinct() 
names.display(5)

In [ ]:
names.drop_duplicates().count()